In [25]:
import pandas as pd
import requests
from datetime import datetime, date

In [3]:
APIKEY = None
with open('apikey.txt', 'r') as _apikey:
    APIKEY = _apikey.read()
    _apikey.close()

https://www.alphavantage.co/documentation/

Example: 

https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220410T0130&limit=100&apikey=

https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220510T0130&topics=finacail_market,earnings&limit=10&apikey=

In [4]:
API_URL_PRICE_DAILY = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&outputsize=full&apikey={APIKEY}"

In [6]:
with requests.Session() as s:
    download = s.get(API_URL_PRICE_DAILY)
    df_price = pd.DataFrame(download.json()['Time Series FX (Daily)']).T
    df_price.columns = [i.split()[1] for i in df_price.columns.values]
    df_price.sort_index(inplace=True)
df_price.to_csv(f"data/APLHAVANTAGE_EURUSD.csv")
df_price

,open,high,low,close
2004-04-02,1.23460,1.23710,1.21300,1.21300
2004-04-05,1.21210,1.21300,1.19790,1.19990
2004-04-06,1.20190,1.21410,1.19790,1.21340
2004-04-07,1.20860,1.21940,1.20440,1.21650
2004-04-08,1.21720,1.22210,1.20570,1.20790
...,...,...,...,...
2023-05-26,1.07278,1.07586,1.07000,1.07250
2023-05-29,1.07210,1.07435,1.07040,1.07088
2023-05-30,1.07088,1.07466,1.06710,1.07325
2023-05-31,1.07319,1.07360,1.06340,1.06856


In [7]:
dates = df_price.index.values
dates_for_news = [i.replace('-','')+'T2350' for i in dates]

In [45]:
link = "https://www.alphavantage.co/query?"
func = "function=NEWS_SENTIMENT"
tickers="tickers=FOREX:EUR"
tickers="tickers=AAPL"
time_from = "time_from=20220310T0130" 
time_to = "time_to=20220410T0130"
url_apikey = "apikey={APIKEY}"

API_URL_NEWS = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOREX:EUR&time_from=20220310T0130&time_to=20220410T0130&apikey={APIKEY}"
API_URL_NEWS = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&time_from=20220310T0130&time_to=20220410T0130&apikey={APIKEY}"


In [46]:
def parse_news_data(_urlNews: str):
    with requests.Session() as s:
        download = s.get(_urlNews)
        df_news = pd.json_normalize(download.json()['feed'])
        # df_news['authors_clear'] = df_news.apply(lambda x: x.authors, axis=1)
    return df_news

In [49]:
df = parse_news_data(API_URL_NEWS)

In [73]:
df_short = df[['title', 'url', 'time_published', 'source_domain']].copy()
df_short['time_published'] = df_short['time_published'].apply(lambda x: datetime.strptime(x, '%Y%m%dT%H%M%S'))
df_short.sort_values(by='time_published', ascending=True, inplace=True)
df_short.reset_index(inplace=True, drop=True)
df_short.head()

,title,url,time_published,source_domain
0,HP Stock Jumps As Warren Buffett Stake Shows V...,https://www.investors.com/news/technology/hpq-...,2022-04-07 16:28:00,www.investors.com
1,"Google In 1996, Apple In 2010: What Does The 1...",https://www.benzinga.com/markets/cryptocurrenc...,2022-04-07 17:28:06,www.benzinga.com
2,"Google In 1996, Apple In 2010: What Does The 1...",https://www.benzinga.com/node/26511633,2022-04-07 17:28:06,www.benzinga.com
3,Here's Why Warren Buffett Bought HPQ Stock,https://www.kiplinger.com/investing/stocks/604...,2022-04-07 18:19:30,www.kiplinger.com
4,10 Stocks That Moved From Thursday's CNBC's 'F...,https://www.benzinga.com/trading-ideas/movers/...,2022-04-07 18:42:22,www.benzinga.com


In [84]:
df_short.source_domain.unique()

array(['www.investors.com', 'www.benzinga.com', 'www.kiplinger.com',
       'www.thestreet.com', 'stockmarket.com', 'www.forbes.com',
       'www.zacks.com', 'www.business-standard.com', 'www.reuters.com',
       'www.marketwatch.com', 'stocknews.com', 'www.fool.com',
       'www.barrons.com', 'www.moneycontrol.com',
       'markets.businessinsider.com'], dtype=object)

In [81]:
test_url = df_short.url[3]
test_url

'https://www.kiplinger.com/investing/stocks/604520/heres-why-warren-buffett-bought-hpq-stock'